# Querying and Creating Portfolios in VAM Client
This notebook serves as a tutorial for querying, creating, and managing portfolios using the `mainsequence.client` package. It covers:
- Querying existing portfolios using filters
- Creating portfolios from time series
- Creating an index asset portfolio
- Querying the newly created index asset portfolio

## Step 1: Import Required Modules
First, we import the necessary modules and constants.

In [7]:
import os
from pathlib import Path
from typing import Dict, Union

import dotenv
# Save the original working directory (only once)
try:
    original_wd
except NameError:
    original_wd = os.getcwd()

# Compute the target directory: two levels up from the original working directory
# For example, if original_wd is /Users/username/project/notebooks,
# then target_dir becomes /Users/username
target_dir = Path(original_wd).parents[2]

# Change the working directory to the target directory
os.chdir(target_dir)
print("Working directory set to:", os.getcwd())

import dotenv
dotenv.load_dotenv('.env')
from mainsequence.client import UpdateStatistics

import mainsequence.client as msc


Working directory set to: /home/jose/code/MainSequenceClientSide/mainsequence-sdk


## Step 2: Query Portfolios
We query an existing portfolio using the `data_node_update_hash_id` filter. This returns a single portfolio object.

In [4]:

# Query a single Portfolio with a specific ticker

portfolio = msc.Portfolio.filter()
print("Queried Portfolio:", portfolio)


2025-08-07T11:22:51.719422Z [debug    ] took 1.7069 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/target_portfolio/ application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at utils.py:107 in make_request())


Queried Portfolio: [Portfolio: 11, Portfolio: 13, Portfolio: 14]


## Step 3: Create Portfolios from Time Series
We create two portfolios for different purposes:
- **Execution Portfolio** for real-time execution
- **Backtesting Portfolio** for simulation and historical analysis

### !!! Important !!!

If you want to start building portfolios we recommend you to use our VirtualFundBuilder. This will help you have a more fluid 
and seamless workflows. In this example we will show you how to build a portfolio without using our Wrapper. A few important considerations

1.  Each portfolio requires a `signal_data_node_update` and a `data_node_update` that should come from TDAG, this is a unique identifier in the backend and placing a random integer will likely collide therefore we need to build a time serie that has a time_index and at least a column with a column "close"



In [23]:
from mainsequence.tdag.data_nodes import DataNode
import numpy as np
import datetime
import pandas as pd

class PortfolioExample(DataNode):
    
    def __init__(self, portfolio_volatility: float,*args, **kwargs):
        self.portfolio_volatility = portfolio_volatility
        super().__init__(*args, **kwargs)


    def dependencies(self) -> Dict[str, Union["DataNode", "APIDataNode"]]:
        return {}

    def update(self):
        """
        Update the time series by simulating a new data point based on the last observation,
        or simulate a complete series if no observations exist.

        :return: pd.DataFrame with index as a timezone-aware datetime (UTC) and a column "close".
        """
        last_observation = self.update_statistics.max_time_index_value
        dt = 1.0  # one day time increment

        mock_asset=msc.Asset.filter(ticker="NVDA")[0]

        if last_observation is not None:
            return pd.DataFrame() # do not make sequential updats for example
          
        else:
            # If no observation exists, simulate a daily series for the last 30 days.
            end_date = datetime.datetime.now(datetime.timezone.utc)
            start_date = end_date - datetime.timedelta(days=30)
            # Generate a date range with daily frequency (timezone-aware in UTC).
            dates = pd.date_range(start=start_date, end=end_date, freq='D', tz=datetime.timezone.utc)
            
            # Initialize with a default price.
            prices = [100.0]
            for _ in range(1, len(dates)):
                random_return = np.random.normal(loc=0, scale=self.portfolio_volatility * np.sqrt(dt))
                new_price = prices[-1] * np.exp(random_return - 0.5 * self.portfolio_volatility**2 * dt)
                prices.append(new_price)
                
            new_data = pd.DataFrame({"close": prices,"rebalance_weights":{mock_asset.unique_identifier:1.0},
                                     "rebalance_price":{mock_asset.unique_identifier:1.0},
                                     "weights_at_last_rebalance":{mock_asset.unique_identifier:1.0},
                                      "price_at_last_rebalance":{mock_asset.unique_identifier:1.0},
                                        "volume_at_last_rebalance":{mock_asset.unique_identifier:1.0},



                                     }, index=dates)
            new_data["last_rebalance_date"]=new_data.reset_index()["index"].apply(lambda x :x.timestamp()).values
            new_data["close_time"]=new_data.reset_index()["index"].apply(lambda x :x.timestamp()).values
            new_data["return"]=new_data["close"].pct_change()
        if last_observation is not None:
            new_data=new_data[new_data.index>self.update_statistics._max_time_in_update_statistics]
         
        return new_data
    
    
portfolio_ts=PortfolioExample(portfolio_volatility=.1)
portfolio_ts.run(debug_mode=True,force_update=True)


2025-08-07T11:29:18.181062Z [info     ] Running post-init routines for PortfolioExample api_time_series=False application_name=ms-sdk command_id=None data_source_id=2 head_local_ts_hash_id=portfolioexample_3a75b432f449e375c40a1ea3486ba9ed job_run_id=None local_hash_id_data_source=2 project_id=1 scheduler_name=DEBUG_177 update_hash=portfolioexample_3a75b432f449e375c40a1ea3486ba9ed (at data_nodes.py:610 in wrapped_init())
2025-08-07T11:29:18.182125Z [info     ] Creating configuration for PortfolioExample api_time_series=False application_name=ms-sdk command_id=None data_source_id=2 head_local_ts_hash_id=portfolioexample_3a75b432f449e375c40a1ea3486ba9ed job_run_id=None local_hash_id_data_source=2 project_id=1 scheduler_name=DEBUG_177 update_hash=portfolioexample_3a75b432f449e375c40a1ea3486ba9ed (at data_nodes.py:638 in _initialize_configuration())
2025-08-07T11:29:18.182907Z [info     ] Post-init routines for PortfolioExample complete. api_time_series=False application_name=ms-sdk command

In [4]:
#we can get our simulated prices from TDAG Backend
print(f"Data in time series",portfolio_ts)
portfolio_ts.get_df_between_dates()

Data in time series PortfolioExample http://192.168.178.69:8000/local-time-series/details/?local_time_serie_id=255


2025-05-19T13:44:01.433937Z [debug    ] took 0.6898 seconds. Requesting POST from http://192.168.178.69:8000/orm/api/ts_manager/local_time_serie/255/get_data_between_dates_from_remote/ api_time_series=False application_name=ms-sdk data_source_id=1 head_local_ts_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac job_run_id=None local_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac local_hash_id_data_source=1 project_id=1 scheduler_name=DEBUG_portfolioexample_f53f240ed0980b8f8bd541172af036ac_1 (at utils.py:95 in make_request())


,close
time_index,
2025-04-19 13:43:50.382000+00:00,100.000000
2025-04-20 13:43:50.382000+00:00,97.834043
2025-04-21 13:43:50.382000+00:00,104.090606
2025-04-22 13:43:50.382000+00:00,95.882577
2025-04-23 13:43:50.382000+00:00,113.441587
2025-04-24 13:43:50.382000+00:00,124.150442
2025-04-25 13:43:50.382000+00:00,108.641368
2025-04-26 13:43:50.382000+00:00,120.532926
2025-04-27 13:43:50.382000+00:00,108.228648


In [25]:

def create_portfolio(portfolio_name,portfolio_ts):
    
    
    existing_portfolio = msc.Portfolio.get_or_none(
       
        data_node_update__id=portfolio_ts.data_node_update.id
    )
    
    if existing_portfolio:
        print(f"Portfolio '{portfolio_name}' already exists.")
        index_asset=msc.PortfolioIndexAsset.get(reference_portfolio__id=existing_portfolio.id)
        
        
        return existing_portfolio,index_asset
    
    
    return msc.Portfolio.create_from_time_series(
        portfolio_name=portfolio_name,
        is_active=True,
        data_node_update_id=portfolio_ts.data_node_update.id,  # Example ID
        signal_data_node_update_id=portfolio_ts.data_node_update.id, #we are not using a signal so we are setting the same
        calendar_name="24/7",
        is_asset_only=False,
        backtest_table_price_column_name="close",
        target_portfolio_about=dict(description= "Test Portfolio For example",
                                    signal_name= "No signal",
                                    signal_description="No description",
                                    rebalance_strategy_name="No rebalance"), timeout=600000
    )

# Create Backtesting Portfolio
backtest_portfolio, portfolio_index_asset = create_portfolio(
  "Example Portfolio",portfolio_ts,)
print("Created Backtest Portfolio:", backtest_portfolio.portfolio_name)


AttributeError: 'PortfolioExample' object has no attribute 'local_metadata'